In [1]:
import torch
import torch.nn as nn
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tqdm
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init

from pipeline.metrics import accuracy_b
from pipeline.model_methods import validate, train
from pipeline.data_loaders import load_img_dataset, load_img_val_dataset

from efficientnet_pytorch import EfficientNet

import timm

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [2]:
class MetaModel(nn.Module):
    def __init__(self, models, device='cuda:0', extended=False):
        super(MetaModel, self).__init__()
        
        self.extended = extended
        self.device = device
        self.models = models
        self.len = len(models)
        
        if self.extended:
            self.bn = nn.BatchNorm1d(self.len)
            self.relu = nn.ReLU()
            self.dropout = nn.Dropout(0.2)

        self.fc = nn.Linear(self.len, 1)
        
    def optimized_forward(self, x):
        x = torch.cat(tuple(x), dim=1)
        
        if self.extended:
            x = self.bn(x)
            x = self.relu(x)
            #x = self.dropout(x)
            
        x = self.fc(x)
        
        return x
        
    def forward(self, x):
        x = torch.cat(tuple([model(x) for model in self.models]), dim=1)
        
        if self.extended:
            x = self.bn(x)
            x = self.relu(x)
            x = self.dropout(x)
            
        x = self.fc(x)
        
        return x

In [3]:
def disable_grad(model):
    for parameter in model.parameters():
        parameter.requires_grad = False
        
    return model

def validate(model, x_val, batch_size, checkpoint=0.31):
    val_loss = []
    dataloader_iterator = iter(x_val)
    model.eval()
    with torch.no_grad():
        for batch_idx in tqdm.tqdm_notebook(range(len(x_val))):
            try:
                X_batch, y_batch, _ = next(dataloader_iterator)
            except:
                dataloader_iterator = iter(x_val)
                X_batch, y_batch, _ = next(dataloader_iterator)

            y_batch = torch.FloatTensor(y_batch).to(device)
            X_batch = X_batch.to(device)

            stacked_preds = []
            preds = []
            
            '''
            for i in range(len(stack_models)):
                stacked_preds.append(stack_models[i](X_batch))

            for i in range(len(models)):
                preds.append(models[i].optimized_forward(stacked_preds[meta_models[i][1]]))
            '''
                
            preds = model(X_batch)
            
            test_loss_value = F.binary_cross_entropy_with_logits(preds, y_batch).item() * batch_size
            #test_loss_value /= len(models)

            val_loss.append(test_loss_value)

        mean_loss = sum(val_loss) / (len(x_val) * batch_size)
        
        if mean_loss <= checkpoint:
            torch.save(model.state_dict(), model.__class__.__name__  + ' ' + str(mean_loss) + '.pth')

        print('Validation: ', mean_loss)
        
        return mean_loss
        
def train(model, x_train, x_val, optimizer, scheduler, batch_size, epochs=10):
    test_loss_history = []

    for epoch in tqdm.tqdm_notebook(range(epochs)):
        dataloader_iterator = iter(x_train)
        model.train()
        
        train_loss = []

        for batch_idx in tqdm.tqdm_notebook(range(len(x_train))):   
            try:
                X_batch, y_batch, _ = next(dataloader_iterator)
            except:
                dataloader_iterator = iter(x_train)
                X_batch, y_batch, _ = next(dataloader_iterator)

            y_batch = torch.FloatTensor(y_batch).to(device)
            X_batch = X_batch.to(device)
            
            optimizer.zero_grad()
            
            stacked_preds = []
            preds = []
            
            '''
            for i in range(len(stack_models)):
                stacked_preds.append(stack_models[i](X_batch))

            for i in range(len(models)):
                preds.append(models[i].optimized_forward(stacked_preds[meta_models[i][1]]))
            '''
                
            preds = model(X_batch)

            loss_value = F.binary_cross_entropy_with_logits(preds, y_batch)
            loss_value.backward()

            optimizer.step()
            #scheduler.step()

            train_loss.append(loss_value.item() * batch_size)

        test_loss_value = validate(model, x_val, batch_size)

        mean_loss = sum(train_loss) / (len(x_train) * batch_size)
        
        print('Train: ', mean_loss)
        print('Epoch:', epoch+1)

        test_loss_history.append(test_loss_value)

    gc.collect()

    return test_loss_history

In [4]:
y_train = pd.read_csv(r'data\metadata.csv')
len(y_train[y_train.label == 1]), len(y_train[y_train.label == 0])

(100000, 19154)

In [5]:
batch_size = 32
x_val = load_img_val_dataset('/workspace/dataset', batch_size)
x_train = load_img_dataset('/workspace/dataset', batch_size, resize=256, crop=None, num_samples=None)

# r'data\img_val
# r'data\train'

RuntimeError: Found 0 files in subfolders of: /workspace/dataset
Supported extensions are: .jpg,.jpeg,.png,.ppm,.bmp,.pgm,.tif,.tiff,.webp

In [6]:
#%%script false

models = []
weights = []

raw_models = \
[
    ['pretrained/efficientnetb2 0.8548137313946486 0.3376769562025044.pth', 'efficientnet-b2'],
    ['pretrained/EfficientNetb3 0.8573518024606384 0.34558522378585194.pth', 'efficientnet-b3'],
    ['pretrained/EfficientNetb4 0.8579110384582294 0.3383911053075265.pth', 'efficientnet-b4'],
    ['pretrained/EfficientNet6 0.8602770369095758 0.33193617861157143.pth', 'efficientnet-b6'],
    
    ['pretrained/EfficientNetb0 t2 0.8616966359803837 0.3698434531609828.pth', 'efficientnet-b0'],
    ['pretrained/EfficientNetb1 t2 0.8410909403768391 0.36058002083572327.pth', 'efficientnet-b1'],
    ['pretrained/EfficientNetb2 t2 0.8659554331928073 0.35598630783834084.pth', 'efficientnet-b2'],
    ['pretrained/EfficientNetb3 t2 0.8486191172674868 0.3611779548592305.pth', 'efficientnet-b3'],
    
    ['pretrained/EfficientNetb3 0.8635894347414609 0.328333642473084.pth', 'efficientnet-b3'],
    ['pretrained/EfficientNetb6 0.8593736556826981 0.32286693639934694.pth', 'efficientnet-b6'],
    
    ['pretrained/tf_efficientnet_b1_ns 0.8571367116923342 0.3341234226295108.pth', 'tf_efficientnet_b1_ns'],
    ['pretrained/tf_efficientnet_b3_ns 0.8712466660930913 0.3277394129117183.pth', 'tf_efficientnet_b3_ns'],
    ['pretrained/tf_efficientnet_b4_ns 0.8708595027101437 0.3152573955405342.pth', 'tf_efficientnet_b4_ns'],
    ['pretrained/tf_efficientnet_b6_ns 0.8733115374688118 0.3156576980666498.pth', 'tf_efficientnet_b6_ns'],
]

meta_models = \
[
    ['pretrained/MetaModel 0.30638167556896007.pth', slice(4, 8), False],
    ['pretrained/MetaModel 0.2919331893755284.pth', slice(0, 4), False],
    ['pretrained/MetaModel 0.30281482560578044.pth', slice(0, 8, None), True],
    ['pretrained/MetaModel 0.26302117601197256.pth', slice(0, 10, None), False],
    ['pretrained/MetaModel 0.264787397152165.pth', slice(0, 14, None), False]
]


stack_models = []
for raw_model in raw_models:
    checkpoint = torch.load(raw_model[0], map_location=device)
    
    if '-' in raw_model[1]:
        model = EfficientNet.from_name(raw_model[1])
        model._fc = nn.Linear(model._fc.in_features, 1)
    else:
        model = timm.create_model(raw_model[1], pretrained=False)
        model.classifier = nn.Linear(model.classifier.in_features, 1)
    
    model.load_state_dict(checkpoint)
    _ = model.eval()
    _ = disable_grad(model)
    model = model.to(device)
    stack_models.append(model)

    del checkpoint, model
    
for meta_raw in meta_models:

    checkpoint = torch.load(meta_raw[0], map_location=device)
    model = MetaModel(models=raw_models[meta_raw[1]], extended=meta_raw[2]).to(device)
    model.load_state_dict(checkpoint)
    _ = model.eval()
    _ = disable_grad(model)
    model.to(device)
    models.append(model)

    del model, checkpoint

In [7]:
epochs = 30

model = MetaModel(stack_models).to(device)

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.) 
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=1e-3, steps_per_epoch=len(x_train), epochs=epochs)

In [8]:
train(model, x_train, x_val, optimizer, scheduler, batch_size, epochs=epochs)